# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
def mask_test_edges_biased_negative_samples(adj, test_percent=10.):
    """
    Task 2: General Biased Negative Samples (B.N.S.) Directed Link
    Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Retrieve all unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj_tilde)

    # Number of positive (= to number of negative) test node pairs to sample
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))

    # Sampling of positive node pairs
    edges_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_idx)
    test_edges_idx = edges_idx[:num_test]
    test_edges = edges_positive[test_edges_idx]

    # In this setting, the reverse node pairs constitute negative samples
    test_edges_false = np.fliplr(test_edges)

    # Get training incomplete adjacency matrix
    train_edges = np.delete(edges_positive, np.hstack([test_edges_idx]), axis = 0)
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    # Validation set: not implemented for Task 2
    # therefore, val_edges and val_edges_false are None
    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0], replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

# 定义模型

In [6]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [7]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

context_embedding_dim = 12
importance_embedding_dim = 48
beta = 5.0
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'Wisconsin'

In [8]:
all_best_auc = []
all_best_ap = []
for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    all_best_ap.append(auc_first_best_ap)
    all_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 198.7703
Epoch: 01, Loss: 198.4052
Epoch: 02, Loss: 197.7263
Epoch: 03, Loss: 196.7906
Epoch: 04, Loss: 195.6555
AP SCORE：  0.4531894836516418
AUC SCORE:  0.400625
Epoch: 00, Loss: 194.3753
Epoch: 01, Loss: 194.3149
Epoch: 02, Loss: 194.2019
Epoch: 03, Loss: 194.0448
Epoch: 04, Loss: 193.8515
AP SCORE：  0.4601971199515824
AUC SCORE:  0.41000000000000003
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 193.6300
Epoch: 01, Loss: 193.3657
Epoch: 02, Loss: 192.8694
Epoch: 03, Loss: 192.1744
Epoch: 04, Loss: 191.3141
AP SCORE：  0.47346478483329474
AUC SCORE:  0.431875
Epoch: 00, Loss: 190.3203
Epoch: 01, Loss: 190.2809
Epoch: 02, Loss: 190.2065
Epoch: 03, Loss: 190.1016
Epoch: 04, Loss: 189.9704
AP SCORE：  0.48124269576971973
AUC SCORE:  0.44375
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 189.8169
Epoch: 01, Loss: 189.5945
Epoch: 02, Loss: 189.1754


Epoch: 04, Loss: 149.2849
AP SCORE：  0.6599623452175191
AUC SCORE:  0.6493749999999999
Epoch: 00, Loss: 148.8398
Epoch: 01, Loss: 148.8040
Epoch: 02, Loss: 148.7362
Epoch: 03, Loss: 148.6399
Epoch: 04, Loss: 148.5185
AP SCORE：  0.6650295040965221
AUC SCORE:  0.6525
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 148.3751
Epoch: 01, Loss: 148.2667
Epoch: 02, Loss: 148.0614
Epoch: 03, Loss: 147.7699
Epoch: 04, Loss: 147.4024
AP SCORE：  0.6691899551363948
AUC SCORE:  0.658125
Epoch: 00, Loss: 146.9681
Epoch: 01, Loss: 146.9323
Epoch: 02, Loss: 146.8643
Epoch: 03, Loss: 146.7679
Epoch: 04, Loss: 146.6465
AP SCORE：  0.684258359725514
AUC SCORE:  0.6625
######################### 模块循环 ： 21 ##########################
Epoch: 00, Loss: 146.5033
Epoch: 01, Loss: 146.3975
Epoch: 02, Loss: 146.1971
Epoch: 03, Loss: 145.9127
Epoch: 04, Loss: 145.5540
AP SCORE：  0.6914192748218708
AUC SCORE:  0.67125
Epoch: 00, Loss: 145.1301
Epoch: 01, Loss: 145.0944
Epoch: 02, Loss: 

Epoch: 03, Loss: 119.4420
Epoch: 04, Loss: 119.3789
AP SCORE：  0.7691824260144834
AUC SCORE:  0.755
######################### 模块循环 ： 39 ##########################
Epoch: 00, Loss: 119.3046
Epoch: 01, Loss: 119.2316
Epoch: 02, Loss: 119.0932
Epoch: 03, Loss: 118.8966
Epoch: 04, Loss: 118.6481
AP SCORE：  0.7705733023814965
AUC SCORE:  0.756875
Epoch: 00, Loss: 118.3541
Epoch: 01, Loss: 118.3361
Epoch: 02, Loss: 118.3020
Epoch: 03, Loss: 118.2537
Epoch: 04, Loss: 118.1931
AP SCORE：  0.7708153458970611
AUC SCORE:  0.7575
######################### 模块循环 ： 40 ##########################
Epoch: 00, Loss: 118.1220
Epoch: 01, Loss: 118.0508
Epoch: 02, Loss: 117.9159
Epoch: 03, Loss: 117.7242
Epoch: 04, Loss: 117.4822
AP SCORE：  0.7702889383023112
AUC SCORE:  0.7575
Epoch: 00, Loss: 117.1959
Epoch: 01, Loss: 117.1788
Epoch: 02, Loss: 117.1465
Epoch: 03, Loss: 117.1007
Epoch: 04, Loss: 117.0433
AP SCORE：  0.7716422118272201
AUC SCORE:  0.759375
######################### 模块循环 ： 41 ##################

Epoch: 03, Loss: 102.0671
Epoch: 04, Loss: 101.9238
AP SCORE：  0.8178575776912442
AUC SCORE:  0.8018749999999999
Epoch: 00, Loss: 101.7542
Epoch: 01, Loss: 101.7460
Epoch: 02, Loss: 101.7306
Epoch: 03, Loss: 101.7086
Epoch: 04, Loss: 101.6810
AP SCORE：  0.8182045164667544
AUC SCORE:  0.8025
######################### 模块循环 ： 59 ##########################
Epoch: 00, Loss: 101.6485
Epoch: 01, Loss: 101.6075
Epoch: 02, Loss: 101.5299
Epoch: 03, Loss: 101.4194
Epoch: 04, Loss: 101.2800
AP SCORE：  0.8193741071100293
AUC SCORE:  0.8031250000000001
Epoch: 00, Loss: 101.1151
Epoch: 01, Loss: 101.1072
Epoch: 02, Loss: 101.0923
Epoch: 03, Loss: 101.0712
Epoch: 04, Loss: 101.0446
AP SCORE：  0.818986475134162
AUC SCORE:  0.803125
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 101.0134
Epoch: 01, Loss: 100.9735
Epoch: 02, Loss: 100.8979
Epoch: 03, Loss: 100.7904
Epoch: 04, Loss: 100.6548
AP SCORE：  0.8207625967942986
AUC SCORE:  0.805
Epoch: 00, Loss: 100.4942
Epoch: 

Epoch: 02, Loss: 91.3298
Epoch: 03, Loss: 91.2514
Epoch: 04, Loss: 91.1522
AP SCORE：  0.8263729222364283
AUC SCORE:  0.81875
Epoch: 00, Loss: 91.0347
Epoch: 01, Loss: 91.0305
Epoch: 02, Loss: 91.0226
Epoch: 03, Loss: 91.0115
Epoch: 04, Loss: 90.9976
AP SCORE：  0.8263729222364283
AUC SCORE:  0.81875
######################### 模块循环 ： 80 ##########################
Epoch: 00, Loss: 90.9813
Epoch: 01, Loss: 90.9526
Epoch: 02, Loss: 90.8981
Epoch: 03, Loss: 90.8207
Epoch: 04, Loss: 90.7228
AP SCORE：  0.8252545011837967
AUC SCORE:  0.818125
Epoch: 00, Loss: 90.6068
Epoch: 01, Loss: 90.6027
Epoch: 02, Loss: 90.5950
Epoch: 03, Loss: 90.5840
Epoch: 04, Loss: 90.5703
AP SCORE：  0.8252545011837967
AUC SCORE:  0.818125
######################### 模块循环 ： 81 ##########################
Epoch: 00, Loss: 90.5541
Epoch: 01, Loss: 90.5259
Epoch: 02, Loss: 90.4723
Epoch: 03, Loss: 90.3960
Epoch: 04, Loss: 90.2997
AP SCORE：  0.8246860672884995
AUC SCORE:  0.816875
Epoch: 00, Loss: 90.1857
Epoch: 01, Loss: 90.1

...

Epoch: 02, Loss: 179.7294
Epoch: 03, Loss: 179.6498
Epoch: 04, Loss: 179.5500
AP SCORE：  0.6657251703248404
AUC SCORE:  0.681875
######################### 模块循环 ： 3 ##########################
Epoch: 00, Loss: 179.4329
Epoch: 01, Loss: 179.2323
Epoch: 02, Loss: 178.8534
Epoch: 03, Loss: 178.3181
Epoch: 04, Loss: 177.6477
AP SCORE：  0.6795504420325372
AUC SCORE:  0.696875
Epoch: 00, Loss: 176.8620
Epoch: 01, Loss: 176.8357
Epoch: 02, Loss: 176.7860
Epoch: 03, Loss: 176.7155
Epoch: 04, Loss: 176.6268
AP SCORE：  0.6876481800665815
AUC SCORE:  0.709375
######################### 模块循环 ： 4 ##########################
Epoch: 00, Loss: 176.5220
Epoch: 01, Loss: 176.3367
Epoch: 02, Loss: 175.9863
Epoch: 03, Loss: 175.4902
Epoch: 04, Loss: 174.8671
AP SCORE：  0.7001026920222289
AUC SCORE:  0.724375
Epoch: 00, Loss: 174.1342
Epoch: 01, Loss: 174.1091
Epoch: 02, Loss: 174.0615
Epoch: 03, Loss: 173.9940
Epoch: 04, Loss: 173.9086
AP SCORE：  0.7148797704922608
AUC SCORE:  0.735
######################### 

Epoch: 04, Loss: 136.0213
AP SCORE：  0.8242540800250647
AUC SCORE:  0.84125
######################### 模块循环 ： 22 ##########################
Epoch: 00, Loss: 135.8935
Epoch: 01, Loss: 135.7832
Epoch: 02, Loss: 135.5744
Epoch: 03, Loss: 135.2783
Epoch: 04, Loss: 134.9053
AP SCORE：  0.8229889722076112
AUC SCORE:  0.839375
Epoch: 00, Loss: 134.4653
Epoch: 01, Loss: 134.4333
Epoch: 02, Loss: 134.3728
Epoch: 03, Loss: 134.2873
Epoch: 04, Loss: 134.1803
AP SCORE：  0.8241993370472768
AUC SCORE:  0.84125
######################### 模块循环 ： 23 ##########################
Epoch: 00, Loss: 134.0549
Epoch: 01, Loss: 133.9478
Epoch: 02, Loss: 133.7448
Epoch: 03, Loss: 133.4569
Epoch: 04, Loss: 133.0943
AP SCORE：  0.8280754762688795
AUC SCORE:  0.84125
Epoch: 00, Loss: 132.6666
Epoch: 01, Loss: 132.6352
Epoch: 02, Loss: 132.5759
Epoch: 03, Loss: 132.4921
Epoch: 04, Loss: 132.3870
AP SCORE：  0.8298032809749558
AUC SCORE:  0.8431249999999999
######################### 模块循环 ： 24 ##########################
Epo

Epoch: 04, Loss: 107.9688
AP SCORE：  0.8237116197076063
AUC SCORE:  0.833125
Epoch: 00, Loss: 107.7117
Epoch: 01, Loss: 107.6954
Epoch: 02, Loss: 107.6647
Epoch: 03, Loss: 107.6214
Epoch: 04, Loss: 107.5672
AP SCORE：  0.8225646759767462
AUC SCORE:  0.83125
######################### 模块循环 ： 42 ##########################
Epoch: 00, Loss: 107.5038
Epoch: 01, Loss: 107.4417
Epoch: 02, Loss: 107.3238
Epoch: 03, Loss: 107.1565
Epoch: 04, Loss: 106.9454
AP SCORE：  0.8237392214572214
AUC SCORE:  0.8325
Epoch: 00, Loss: 106.6959
Epoch: 01, Loss: 106.6807
Epoch: 02, Loss: 106.6520
Epoch: 03, Loss: 106.6115
Epoch: 04, Loss: 106.5608
AP SCORE：  0.8220337430276052
AUC SCORE:  0.83125
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 106.5017
Epoch: 01, Loss: 106.4414
Epoch: 02, Loss: 106.3271
Epoch: 03, Loss: 106.1648
Epoch: 04, Loss: 105.9602
AP SCORE：  0.824811520805383
AUC SCORE:  0.831875
Epoch: 00, Loss: 105.7182
Epoch: 01, Loss: 105.7040
Epoch: 02, Loss: 105.6772


Epoch: 04, Loss: 91.8903
AP SCORE：  0.7967498563275526
AUC SCORE:  0.8093750000000001
Epoch: 00, Loss: 91.7373
Epoch: 01, Loss: 91.7294
Epoch: 02, Loss: 91.7143
Epoch: 03, Loss: 91.6930
Epoch: 04, Loss: 91.6662
AP SCORE：  0.7962282430700902
AUC SCORE:  0.80875
######################### 模块循环 ： 63 ##########################
Epoch: 00, Loss: 91.6344
Epoch: 01, Loss: 91.5974
Epoch: 02, Loss: 91.5272
Epoch: 03, Loss: 91.4274
Epoch: 04, Loss: 91.3012
AP SCORE：  0.7968604269781362
AUC SCORE:  0.809375
Epoch: 00, Loss: 91.1518
Epoch: 01, Loss: 91.1441
Epoch: 02, Loss: 91.1296
Epoch: 03, Loss: 91.1089
Epoch: 04, Loss: 91.0829
AP SCORE：  0.7972797483974576
AUC SCORE:  0.809375
######################### 模块循环 ： 64 ##########################
Epoch: 00, Loss: 91.0523
Epoch: 01, Loss: 91.0161
Epoch: 02, Loss: 90.9475
Epoch: 03, Loss: 90.8500
Epoch: 04, Loss: 90.7268
AP SCORE：  0.7981689346300758
AUC SCORE:  0.810625
Epoch: 00, Loss: 90.5808
Epoch: 01, Loss: 90.5734
Epoch: 02, Loss: 90.5593
Epoch: 03,

Epoch: 01, Loss: 82.3100
Epoch: 02, Loss: 82.2603
Epoch: 03, Loss: 82.1895
Epoch: 04, Loss: 82.1000
AP SCORE：  0.7821034390994129
AUC SCORE:  0.799375
Epoch: 00, Loss: 81.9939
Epoch: 01, Loss: 81.9879
Epoch: 02, Loss: 81.9765
Epoch: 03, Loss: 81.9602
Epoch: 04, Loss: 81.9397
AP SCORE：  0.7821034390994129
AUC SCORE:  0.799375
######################### 模块循环 ： 83 ##########################
Epoch: 00, Loss: 81.9154
Epoch: 01, Loss: 81.8895
Epoch: 02, Loss: 81.8405
Epoch: 03, Loss: 81.7708
Epoch: 04, Loss: 81.6826
AP SCORE：  0.7818690640994129
AUC SCORE:  0.79875
Epoch: 00, Loss: 81.5780
Epoch: 01, Loss: 81.5720
Epoch: 02, Loss: 81.5605
Epoch: 03, Loss: 81.5443
Epoch: 04, Loss: 81.5239
AP SCORE：  0.7816417913721402
AUC SCORE:  0.798125
######################### 模块循环 ： 84 ##########################
Epoch: 00, Loss: 81.4997
Epoch: 01, Loss: 81.4742
Epoch: 02, Loss: 81.4259
Epoch: 03, Loss: 81.3571
Epoch: 04, Loss: 81.2701
AP SCORE：  0.7814213029053831
AUC SCORE:  0.7975
Epoch: 00, Loss: 81.16

In [9]:
print("MEAN AP : ", np.array(all_best_ap).mean())
print("STD AP : ", np.array(all_best_ap).std())
print("MEAN AUC : ", np.array(all_best_auc).mean())
print("STD AUC : ", np.array(all_best_auc).std())
"""
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)
"""

MEAN AP :  0.8081003606527339
STD AP :  0.050127766466642146
MEAN AUC :  0.8194999999999999
STD AUC :  0.03683663258225435


'\nprint("AUC FIRST BEST AUC: ", auc_first_best_auc)\nprint("AUC FIRST BEST AP: ", auc_first_best_ap)\nprint("AP FIRST BEST AUC: ", ap_first_best_auc)\nprint("AP FIRST BEST AP: ", ap_first_best_ap)\n'